In [93]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
%matplotlib inline

In [94]:
import torch
from torchvision import datasets, models, transforms
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torchvision.datasets import ImageFolder
import os
from torch.optim import lr_scheduler
from torch.optim.lr_scheduler import ReduceLROnPlateau
import pretrainedmodels
from adamW import AdamW
from lr_finder import LRFinder

In [95]:
!ls lrdata/coffee_table/style/train

Asian	      Craftsman   Mediterranean  Rustic        Tropical
Beach Style   Farmhouse   Midcentury	 Traditional   Victorian
Contemporary  Industrial  Modern	 Transitional


In [112]:
#Example code for only one of my attribute training sets
path = 'Osamadata/basecolorboth'

In [15]:
from PIL import Image
#NOTE: CHANGE THIS IF YOU WANT TO SWITCH DIR
#removes images from training set that are corrupt.
images_path = 'Osamadata/basecolor/valid'
image_extensions = ['.jpg', '.png', '.jpeg']
images = [os.path.join(dp, f) for dp, dn, filenames in os.walk(images_path)
         for f in filenames if os.path.splitext(f)[1].lower() in image_extensions]

for i, item in enumerate(images):
    try:
        im = Image.open(item)
    except IOError:
        os.remove(item)
        print('removed '+ item)

In [60]:
#this is necessary because all of the images in my training set are symbolic links
def loadsymlink(path):
    returnImage = Image.open(os.readlink(path))
    return returnImage.convert('RGB')

In [113]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

data_transforms = {
    'train':
        transforms.Compose([
            transforms.Resize((224,224)),
            transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            normalize
        ]),
    'valid':
        transforms.Compose([
            transforms.Resize((224,224)),
            transforms.ToTensor(),
            normalize
        ])
}

image_datasets = {
    'train':
        ImageFolder(os.path.join(path,'train'), data_transforms['train']),
    'valid':
        ImageFolder(os.path.join('Osamadata/basecolorboth','valid'), data_transforms['valid'])}

dataloaders = {
    'train':
        torch.utils.data.DataLoader(
            image_datasets['train'],
            batch_size=8,
            shuffle=True,
            num_workers=4),
    'valid':
        torch.utils.data.DataLoader(
            image_datasets['valid'],
            batch_size=8,
            shuffle=False,
            num_workers=4)}

In [114]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = models.resnet50(pretrained=True).to(device)

#good LR always happens to be small for AdamW
optimizer = AdamW(model.parameters(), lr=1e-5, weight_decay=1e-4)
#optimizer = optim.SGD(model.parameters(), lr=1e-2, weight_decay=1e-4)

num_ftrs = model.fc.in_features
model.fc = torch.nn.Linear(num_ftrs, 16).to(device)

criterion = nn.CrossEntropyLoss()

print("creating learning rate scheduler")


steps = len(dataloaders['train'])
exp_lr_scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, steps)

creating learning rate scheduler


In [32]:
#typically I get my best lr right before the descent in loss, make sure to set lr = 1e-8 before running
lr_finder = LRFinder(model, optimizer, criterion, device='cuda')
lr_finder.range_test(dataloaders['train'], end_lr=1, num_iter=100)
lr_finder.plot()

In [115]:
def train_model(model, criterion, optimizer, dataloaders, scheduler=None, num_epochs=20,cycle_mult=1):

    steps = len(dataloaders['train'])
    completed = 0
    
    best_acc = 0.0
    
    dataset_sizes = {'train':len(dataloaders['train'].dataset),'valid':len(dataloaders['valid'].dataset)}
    
    #wandb.watch(model)
    
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch+1, num_epochs))
        print('-' * 10)
 
        epoch_loss = 1.0

        for phase in ['train', 'valid']:
            if phase == 'train':
                model.train()
            else:
                model.eval()
 
            running_loss = 0.0
            running_corrects = 0
 
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)
 
                outputs = model(inputs)
                loss = criterion(outputs, labels)
 
                if phase == 'train':
                    if scheduler is not None:
                        steps -= 1
                        scheduler.step()
                    optimizer.zero_grad()
                    loss.backward()
                    optimizer.step()
                    
 
                _, preds = torch.max(outputs, 1)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
 
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
        
            if phase == 'valid':
                #wandb.log({'epoch_loss':epoch_loss,'epoch_acc':epoch_acc})
                
                if epoch_acc > best_acc:
                    best_acc = epoch_acc
                    best_model_wts = model.state_dict()
        
        
        if scheduler is not None:
            lrval = scheduler.get_lr()[0]
            print('steps: %d lr: %.8f'%(steps,lrval))
            
        if (steps <= 0):
            steps = len(dataloaders['train'])
            print(steps)
            scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, steps)

        print('{} loss: {:.4f}, acc: {:.4f}'.format(phase,
                                                    epoch_loss,
                                                    epoch_acc))    
    
    #torch.save(best_model_wts, 'testsave.pt')
    return model

In [79]:
#shade shape
#product bv
model_trained = train_model(model, criterion, optimizer, dataloaders, scheduler=None, num_epochs=75, cycle_mult=2)

Epoch 1/75
----------
valid loss: 2.5319, acc: 0.3023
Epoch 2/75
----------
valid loss: 2.1013, acc: 0.4884
Epoch 3/75
----------
valid loss: 1.6923, acc: 0.5116
Epoch 4/75
----------
valid loss: 1.5440, acc: 0.5581
Epoch 5/75
----------
valid loss: 1.4170, acc: 0.5698
Epoch 6/75
----------
valid loss: 1.3801, acc: 0.5581
Epoch 7/75
----------
valid loss: 1.3727, acc: 0.5465
Epoch 8/75
----------
valid loss: 1.3250, acc: 0.5581
Epoch 9/75
----------
valid loss: 1.2556, acc: 0.6395
Epoch 10/75
----------
valid loss: 1.2574, acc: 0.6279
Epoch 11/75
----------
valid loss: 1.2373, acc: 0.6163
Epoch 12/75
----------
valid loss: 1.2688, acc: 0.6512
Epoch 13/75
----------
valid loss: 1.2811, acc: 0.6744
Epoch 14/75
----------
valid loss: 1.3502, acc: 0.6512
Epoch 15/75
----------
valid loss: 1.2682, acc: 0.6279
Epoch 16/75
----------
valid loss: 1.3471, acc: 0.6628
Epoch 17/75
----------
valid loss: 1.3744, acc: 0.6860
Epoch 18/75
----------
valid loss: 1.2719, acc: 0.6628
Epoch 19/75
-------

In [84]:
#product and cropped bv
model_trained = train_model(model, criterion, optimizer, dataloaders, scheduler=None, num_epochs=75, cycle_mult=2)

Epoch 1/75
----------
valid loss: 2.6173, acc: 0.2326
Epoch 2/75
----------
valid loss: 2.1130, acc: 0.4186
Epoch 3/75
----------
valid loss: 1.8152, acc: 0.4767
Epoch 4/75
----------
valid loss: 1.6445, acc: 0.5233
Epoch 5/75
----------
valid loss: 1.3787, acc: 0.5698
Epoch 6/75
----------
valid loss: 1.3809, acc: 0.5349
Epoch 7/75
----------
valid loss: 1.2985, acc: 0.6047
Epoch 8/75
----------
valid loss: 1.2065, acc: 0.6047
Epoch 9/75
----------
valid loss: 1.2213, acc: 0.5814
Epoch 10/75
----------
valid loss: 1.2174, acc: 0.6512
Epoch 11/75
----------
valid loss: 1.1770, acc: 0.6279
Epoch 12/75
----------
valid loss: 1.2113, acc: 0.6512
Epoch 13/75
----------
valid loss: 1.1102, acc: 0.6512
Epoch 14/75
----------
valid loss: 1.1583, acc: 0.6628
Epoch 15/75
----------
valid loss: 1.1462, acc: 0.6977
Epoch 16/75
----------
valid loss: 1.1311, acc: 0.6860
Epoch 17/75
----------
valid loss: 1.1486, acc: 0.6860
Epoch 18/75
----------
valid loss: 1.1309, acc: 0.6860
Epoch 19/75
-------

In [100]:
#base material
#product both valid
model_trained = train_model(model, criterion, optimizer, dataloaders, scheduler=None, num_epochs=75, cycle_mult=2)

Epoch 1/75
----------
valid loss: 1.8174, acc: 0.3704
Epoch 2/75
----------
valid loss: 1.5966, acc: 0.4907
Epoch 3/75
----------
valid loss: 1.4428, acc: 0.5185
Epoch 4/75
----------
valid loss: 1.3585, acc: 0.5370
Epoch 5/75
----------
valid loss: 1.2804, acc: 0.5648
Epoch 6/75
----------
valid loss: 1.2480, acc: 0.6019
Epoch 7/75
----------
valid loss: 1.1942, acc: 0.6296
Epoch 8/75
----------
valid loss: 1.1834, acc: 0.6204
Epoch 9/75
----------
valid loss: 1.1695, acc: 0.6481
Epoch 10/75
----------
valid loss: 1.1655, acc: 0.6296
Epoch 11/75
----------
valid loss: 1.1198, acc: 0.6389
Epoch 12/75
----------
valid loss: 1.1900, acc: 0.6481
Epoch 13/75
----------
valid loss: 1.1372, acc: 0.6481
Epoch 14/75
----------
valid loss: 1.1947, acc: 0.6481
Epoch 15/75
----------
valid loss: 1.2451, acc: 0.6574
Epoch 16/75
----------
valid loss: 1.2125, acc: 0.6759
Epoch 17/75
----------
valid loss: 1.2383, acc: 0.6759
Epoch 18/75
----------
valid loss: 1.3596, acc: 0.6389
Epoch 19/75
-------

In [105]:
#Product and cropped both valid
model_trained = train_model(model, criterion, optimizer, dataloaders, scheduler=None, num_epochs=75, cycle_mult=2)

Epoch 1/75
----------
valid loss: 1.8421, acc: 0.2963
Epoch 2/75
----------
valid loss: 1.5757, acc: 0.5278
Epoch 3/75
----------
valid loss: 1.4078, acc: 0.5926
Epoch 4/75
----------
valid loss: 1.3112, acc: 0.6296
Epoch 5/75
----------
valid loss: 1.1987, acc: 0.6574
Epoch 6/75
----------
valid loss: 1.1469, acc: 0.6389
Epoch 7/75
----------
valid loss: 1.1283, acc: 0.6481
Epoch 8/75
----------
valid loss: 1.0946, acc: 0.6481
Epoch 9/75
----------
valid loss: 1.0880, acc: 0.6296
Epoch 10/75
----------
valid loss: 1.0404, acc: 0.6852
Epoch 11/75
----------
valid loss: 1.0528, acc: 0.6667
Epoch 12/75
----------
valid loss: 1.0511, acc: 0.6667
Epoch 13/75
----------
valid loss: 0.9812, acc: 0.6944
Epoch 14/75
----------
valid loss: 1.0257, acc: 0.6944
Epoch 15/75
----------
valid loss: 1.0349, acc: 0.7037
Epoch 16/75
----------
valid loss: 1.0224, acc: 0.7037
Epoch 17/75
----------
valid loss: 1.0740, acc: 0.7037
Epoch 18/75
----------
valid loss: 1.0872, acc: 0.6944
Epoch 19/75
-------

In [111]:
#basecolor
#product
model_trained = train_model(model, criterion, optimizer, dataloaders, scheduler=None, num_epochs=75, cycle_mult=2)

Epoch 1/75
----------
valid loss: 2.3914, acc: 0.2587
Epoch 2/75
----------
valid loss: 1.9286, acc: 0.3943
Epoch 3/75
----------
valid loss: 1.7718, acc: 0.4353
Epoch 4/75
----------
valid loss: 1.5970, acc: 0.4953
Epoch 5/75
----------
valid loss: 1.5243, acc: 0.5079
Epoch 6/75
----------
valid loss: 1.5334, acc: 0.5047
Epoch 7/75
----------
valid loss: 1.4986, acc: 0.5205
Epoch 8/75
----------
valid loss: 1.4608, acc: 0.5142
Epoch 9/75
----------
valid loss: 1.5164, acc: 0.5110
Epoch 10/75
----------
valid loss: 1.4844, acc: 0.5237
Epoch 11/75
----------
valid loss: 1.4885, acc: 0.5047
Epoch 12/75
----------
valid loss: 1.5529, acc: 0.5110
Epoch 13/75
----------
valid loss: 1.5394, acc: 0.5552
Epoch 14/75
----------
valid loss: 1.6024, acc: 0.5615
Epoch 15/75
----------
valid loss: 1.6133, acc: 0.5142
Epoch 16/75
----------
valid loss: 1.6340, acc: 0.5331
Epoch 17/75
----------
valid loss: 1.7628, acc: 0.4953
Epoch 18/75
----------
valid loss: 1.6814, acc: 0.5331
Epoch 19/75
-------

In [116]:
#product + cropped
model_trained = train_model(model, criterion, optimizer, dataloaders, scheduler=None, num_epochs=75, cycle_mult=2)

Epoch 1/75
----------
valid loss: 2.2734, acc: 0.2934
Epoch 2/75
----------
valid loss: 1.8619, acc: 0.4416
Epoch 3/75
----------
valid loss: 1.6164, acc: 0.4953
Epoch 4/75
----------
valid loss: 1.4970, acc: 0.4984
Epoch 5/75
----------
valid loss: 1.4498, acc: 0.5552
Epoch 6/75
----------
valid loss: 1.3813, acc: 0.5363
Epoch 7/75
----------
valid loss: 1.3893, acc: 0.5300
Epoch 8/75
----------
valid loss: 1.3698, acc: 0.5394
Epoch 9/75
----------
valid loss: 1.3737, acc: 0.5394
Epoch 10/75
----------
valid loss: 1.3443, acc: 0.5552
Epoch 11/75
----------
valid loss: 1.2945, acc: 0.5710
Epoch 12/75
----------
valid loss: 1.3360, acc: 0.5741
Epoch 13/75
----------
valid loss: 1.3358, acc: 0.6025
Epoch 14/75
----------
valid loss: 1.3907, acc: 0.5741
Epoch 15/75
----------
valid loss: 1.4034, acc: 0.5741
Epoch 16/75
----------
valid loss: 1.4503, acc: 0.5647
Epoch 17/75
----------
valid loss: 1.4251, acc: 0.5836
Epoch 18/75
----------
valid loss: 1.4629, acc: 0.5710
Epoch 19/75
-------